In [ ]:
import numpy  as np
import pandas as pd
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from stop_words import get_stop_words
from nltk.tokenize import RegexpTokenizer
import unicodedata

pd.set_option('display.max_colwidth', 30)

import pickle

In [5]:
md = pd.read_pickle("E:/Data/MIR/EU_SOUNDS_FEATURES/combined_metadata.p")

In [6]:
md.ix[221100]

#allsound; 400615                       http://econnect.ait.co.at/...
index                                                          236221
title                                              San José De Flores
aboutaggregations.0.about                                        None
aggregations.0.edmDataProvider.def                            Pattaya
aggregations.0.edmProvider.def                                DISMARC
aggregations.0.edmRights.def            http://www.europeana.eu/ri...
aggregations.0.webResources.0.about     http://econnect.ait.co.at/...
aggregations.0.edmIsShownBy             http://econnect.ait.co.at/...
proxies.0.dctermsCreated.def                                     None
proxies.0.dctermsExtent.def             00:03:33, 00:00:30 (previe...
proxies.0.dctermsMedium.def                                      None
proxies.0.dctermsSpatial.def                                     None
proxies.0.dctermsTOC.def                                         None
proxies.0.dctermsIsP

In [7]:
documents = (((((
(md["proxies.0.dcCreator.def"] + " ")
.str.cat(md["proxies.0.dcContributor.def"]) + " ")
.str.cat(md["title"]) + " ")
.str.cat(md["proxies.0.dcDescription.def"]) + " ")
.str.cat(md["proxies.0.dctermsSpatial.def"]) + " ")
.str.cat(md["proxies.0.dcSubject.def"])
.str.strip()).fillna("")

In [8]:
DATA_PATH = "G:/eu_sounds_challenge/"

In [ ]:
# read csv-data (separated by semicolons)
metadata = pd.read_csv("E:/Features/MIR/EuropeanaSounds/metadata/metadata.csv", sep=";", encoding="utf-8")

# convert nan-values to empty strings
metadata = metadata.fillna("")

# use the "id" field as index
metadata = metadata.set_index("id")

In [10]:
metadata["genre"] = ""

In [11]:
pd.set_option('display.max_colwidth', 50)
metadata.ix[1110]

contributor     Šmits, Herberts, Vētra, Mariss, 1901-1965
country                                            latvia
created                                                  
creator                                                  
dataProvider                   National Library of Latvia
date                                               [1935]
description                                              
format             Shellac record, audio/mpeg, image/jpeg
identifier                                           2623
language                                                 
medium                                         audio disc
provider                                 Europeana Sounds
publisher                              Latvija : Homophon
relation                                                 
spatial                                                  
subject                           Popular music -- Latvia
title                           Vecais ratiņš : slov-foks
type          

In [12]:
documents = ((((((
(metadata.genre + " ")
.str.cat(metadata.creator) + " ")
.str.cat(metadata.contributor) + " ")
.str.cat(metadata.title) + " ")
.str.cat(metadata.description) + " ")
.str.cat(metadata.subject) + " ")
.str.cat(metadata.country)
.str.strip()).values

In [13]:
stop_words = []
stop_words.extend(get_stop_words('en'))
stop_words.extend(get_stop_words('de'))
stop_words.extend(get_stop_words('fr'))
stop_words.extend(get_stop_words('it'))
stop_words.extend(get_stop_words('pt'))
stop_words.extend(get_stop_words('ro'))
stop_words.extend(get_stop_words('spanish'))

In [14]:
tokenizer = RegexpTokenizer(r'\w+')

texts = [[unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore') 
              for word in tokenizer.tokenize(document.lower()) 
                  if (word not in stop_words and len(word) > 1)] for document in documents if len(document) > 0]

In [15]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if (frequency[token] > 1)]
         for text in texts]

In [499]:
documents[220200]

''

In [493]:
len(documents)

312096

In [491]:
" ".join(texts[22000])

IndexError: list index out of range

In [443]:
np.mean([len(texts[i]) for i in range(len(texts))])

43.317685321551302

In [444]:
model = gensim.models.Word2Vec(texts, size=1000, window=10, min_count=5, workers=4)

2016-10-19 14:12:24,276 : INFO : collecting all words and their counts
2016-10-19 14:12:24,278 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-10-19 14:12:24,322 : INFO : PROGRESS: at sentence #10000, processed 305429 words, keeping 7972 word types
2016-10-19 14:12:24,367 : INFO : PROGRESS: at sentence #20000, processed 620071 words, keeping 12477 word types
2016-10-19 14:12:24,479 : INFO : PROGRESS: at sentence #30000, processed 1312027 words, keeping 39732 word types
2016-10-19 14:12:24,496 : INFO : collected 40226 word types from a corpus of 1411810 raw words and 32592 sentences
2016-10-19 14:12:24,565 : INFO : min_count=5 retains 19368 unique words (drops 20858)
2016-10-19 14:12:24,566 : INFO : min_count leaves 1357258 word corpus (96% of original 1411810)
2016-10-19 14:12:24,644 : INFO : deleting the raw counts dictionary of 40226 items
2016-10-19 14:12:24,647 : INFO : sample=0.001 downsamples 30 most-common words
2016-10-19 14:12:24,647 : INFO : do

In [294]:
model.similarity("mozart", "amadeus")

0.98782684219615091

In [295]:
model.n_similarity(["chopin","offenbach"],["classic"])

0.76947588192284821

In [500]:
model.most_similar(["majeur", "major", "andante", "allegro", "sonata", "fugue", "sonate", "compositeur"], 
                   ["singer", "arrangeur"],
                   topn=30)

[('core', 0.9724166393280029),
 ('alice', 0.9718669652938843),
 ('pacini', 0.9699550867080688),
 ('htb', 0.9669064283370972),
 ('mazzoleni', 0.9667044878005981),
 ('scarlatti', 0.9662805795669556),
 ('clavicorde', 0.9662434458732605),
 ('vlc', 0.9657636284828186),
 ('caruso', 0.965447723865509),
 ('giovanni', 0.9652591943740845),
 ('fl', 0.9651468396186829),
 ('lombardi', 0.9647630453109741),
 ('alma', 0.9643343687057495),
 ('castellazzi', 0.9630222320556641),
 ('arioso', 0.9626851081848145),
 ('pagliacci', 0.9626724720001221),
 ('gurlitt', 0.9615856409072876),
 ('torna', 0.9615195989608765),
 ('prelude', 0.9615018367767334),
 ('prologo', 0.9613460898399353),
 ('basson', 0.9613147974014282),
 ('bodky', 0.9596811532974243),
 ('strofe', 0.9595828652381897),
 ('seville', 0.9594621658325195),
 ('mattia', 0.9594092965126038),
 ('vidi', 0.9592790007591248),
 ('gluck', 0.9583433866500854),
 ('arr', 0.9575711488723755),
 ('russ', 0.9575594663619995),
 ('concerto', 0.9573565721511841)]

In [446]:
model.most_similar_cosmul(["concerto", "prelude", "duett", "fugue"], 
                          [], topn=30)

[('castellazzi', 0.9378741979598999),
 ('caruso', 0.9192914366722107),
 ('giovanni', 0.9192336797714233),
 ('patti', 0.9189188480377197),
 ('gluck', 0.9147874116897583),
 ('arioso', 0.9143836498260498),
 ('htb', 0.9141067862510681),
 ('adelina', 0.9128225445747375),
 ('bodky', 0.9102062582969666),
 ('ulysses', 0.9095026850700378),
 ('terra', 0.9091941118240356),
 ('alice', 0.9085679650306702),
 ('terzetto', 0.9083512425422668),
 ('johan', 0.9078710675239563),
 ('zenatello', 0.9078000783920288),
 ('strofe', 0.9076767563819885),
 ('russ', 0.905815601348877),
 ('pacini', 0.9050401449203491),
 ('mefistofele', 0.9033681154251099),
 ('cavatina', 0.9033201336860657),
 ('giannina', 0.9023322463035583),
 ('core', 0.9014116525650024),
 ('soave', 0.9009794592857361),
 ('gurlitt', 0.8999053835868835),
 ('fl', 0.8995052576065063),
 ('lombardi', 0.898522675037384),
 ('vidi', 0.8971827626228333),
 ('rossini', 0.8966554999351501),
 ('prologo', 0.8937667012214661),
 ('parsi', 0.8925209045410156)]

In [520]:
model.most_similar_cosmul(positive=["wolfgang", "amadeus", "mozart",
                                    "friedrich", "handel", 
                                    "compositeur"], topn=20)

[('1791', 0.8946651220321655),
 ('1756', 0.8765036463737488),
 ('1759', 0.8638619780540466),
 ('1685', 0.8564209342002869),
 ('christoph', 0.8558024764060974),
 ('gaetano', 0.837415874004364),
 ('1843', 0.8364599347114563),
 ('1714', 0.8359813094139099),
 ('alda', 0.8348263502120972),
 ('willibald', 0.8292129039764404),
 ('giuseppe', 0.8281602263450623),
 ('carl', 0.8261173963546753),
 ('1801', 0.8252905011177063),
 ('1886', 0.8222440481185913),
 ('schubert', 0.8219956755638123),
 ('sebastian', 0.8212117552757263),
 ('gustav', 0.8193891644477844),
 ('1896', 0.8149348497390747),
 ('stransky', 0.810939371585846),
 ('gioachino', 0.810754656791687)]

In [358]:
for x in model.most_similar(positive=["composer", "compositeur", "smetana", "bach", "beethoven", "handel", "wagner", "verdi", "rossini"],
                            negative=["lyricist", "singer", "instrumentalist", "arranger", "informateur", "flute"],
                            topn=20):
    print x[0], x[1]

giuseppe 0.919923424721
av 0.916781008244
giacomo 0.913340687752
van 0.912716984749
vincenzo 0.91206240654
gentil 0.909741520882
xavier 0.906051695347
faust 0.904570698738
luisa 0.897276759148
franz 0.897090554237
iv 0.895191133022
lucia 0.892682552338
deh 0.88970053196
strauss 0.889042615891
vieni 0.888922393322
med 0.886771559715
orchester 0.886404752731
rockstroh 0.884919524193
gaetano 0.884215593338
freischutz 0.883634269238


In [13]:
print len(model.vocab)
#model.vocab

19367


In [359]:
bigram = gensim.models.Phrases(texts)

2016-10-19 13:44:50,095 : INFO : collecting all words and their counts
2016-10-19 13:44:50,096 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2016-10-19 13:44:51,177 : INFO : PROGRESS: at sentence #10000, processed 282265 words and 34314 word types
2016-10-19 13:44:52,437 : INFO : PROGRESS: at sentence #20000, processed 578896 words and 64822 word types
2016-10-19 13:44:55,247 : INFO : PROGRESS: at sentence #30000, processed 1230777 words and 396403 word types
2016-10-19 13:44:55,637 : INFO : collected 414253 word types from a corpus of 1320847 words (unigram + bigrams) and 32592 sentences
2016-10-19 13:44:55,638 : INFO : merging 414253 counts into Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2016-10-19 13:44:55,871 : INFO : merged Phrases<414253 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [360]:
print(bigram[texts[0]])

[u'schorr', u'lyricist', u'burstein_peisach', u'singer', u'secunda', u'composer', u'voice_solo', u'france']


In [361]:
texts_bigram = [bigram[text] for text in texts]

In [362]:
model_bigram = gensim.models.Word2Vec(texts_bigram, size=1000, window=20, min_count=5, workers=4)

2016-10-19 13:46:02,096 : INFO : collecting all words and their counts
2016-10-19 13:46:02,098 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-10-19 13:46:02,142 : INFO : PROGRESS: at sentence #10000, processed 209584 words, keeping 8731 word types
2016-10-19 13:46:02,193 : INFO : PROGRESS: at sentence #20000, processed 399440 words, keeping 16134 word types
2016-10-19 13:46:02,334 : INFO : PROGRESS: at sentence #30000, processed 936763 words, keeping 48676 word types
2016-10-19 13:46:02,354 : INFO : collected 49575 word types from a corpus of 1004625 raw words and 32592 sentences
2016-10-19 13:46:02,444 : INFO : min_count=5 retains 25881 unique words (drops 23694)
2016-10-19 13:46:02,446 : INFO : min_count leaves 943225 word corpus (93% of original 1004625)
2016-10-19 13:46:02,551 : INFO : deleting the raw counts dictionary of 49575 items
2016-10-19 13:46:02,552 : INFO : sample=0.001 downsamples 25 most-common words
2016-10-19 13:46:02,553 : INFO : down

In [19]:
df = pd.DataFrame(list(model_bigram.vocab))
df[df[0].str.contains("mozart")]

,0
9186,orchestre_mozart
14492,academica_mozarteum
15263,mozart
17532,mozart_comp
21108,mozart_wolfgang
21778,divertissement_mozart


In [363]:
for x in model_bigram.most_similar(positive=["irish", "folk", "ireland", "irish_traditional", "flogging_tune", "sheridan", 
                                             "lough_erne", "herbert_hughes", "seamus"], 
                                   #negative=["flamenco","paris","french", "ukrainian"], 
                                   topn=20):
    print x[0], x[1]

2016-10-19 13:46:29,895 : INFO : precomputing L2-norms of word weight vectors


joe_cooley 0.978929936886
mrs 0.976350128651
unidentified 0.974154114723
harrison 0.973539113998
frank_quinn 0.970666527748
unidentified_piano 0.969449281693
william_mullaly 0.968471765518
concert_hall 0.967836141586
guitar_instrumentalist 0.967309653759
island 0.966392278671
market 0.966380953789
kesh 0.965722203255
micheal 0.965506672859
tenpenny 0.965464949608
tim 0.96447789669
fiddle_unidentified 0.964016854763
lisdoon 0.963634133339
murty_rabbett 0.963227391243
seoighe 0.962851941586
newly_mown 0.962595105171


In [375]:
for x in model_bigram.most_similar(positive=["beethoven", "compositeur"], 
                                   #negative=["sharpe", "soprano"], 
                                   topn=20):
    print x[0], x[1]

friedrich_compositeur 0.997922480106
handel_georg 0.996378064156
sharpe 0.995897531509
boulanger_ensemble 0.995296239853
sv 0.994497299194
gavotte 0.99421620369
orch_comprend 0.994091629982
rameau_comp 0.993809700012
donateur_anonyme 0.992979943752
compositeur_rouget 0.992966294289
nadia_boulanger 0.992602467537
mozart_comp 0.992157936096
pascal_seguin 0.992108225822
generosite 0.991973340511
arr 0.991805076599
soliste_france 0.991728007793
orchestre_comprend 0.991655170918
soprano 0.991555035114
meistersinger 0.991267800331
ensemble_nadia 0.99123442173


documents = ((((
(metadata.contributor + " ")
.str.cat(metadata.title) + " ")
.str.cat(metadata.description) + " ")
.str.cat(metadata.subject) + " ")
.str.cat(metadata.country)
.str.strip()).values

In [ ]:
"smetana", "bach", "beethoven", "handel", "wagner", "verdi", "rossini", "mozart"

In [286]:
pd.set_option('display.max_colwidth', 300)
search = "moynagh"

metadata[
metadata["contributor"].str.lower().str.contains(search) | 
metadata["creator"].str.lower().str.contains(search) |
metadata["title"].str.lower().str.contains(search) |
metadata["subject"].str.lower().str.contains(search) |
metadata["description"].str.lower().str.contains(search) 
].shape

(5, 20)

In [287]:
metadata.loc[
metadata["contributor"].str.lower().str.contains(search) | 
metadata["creator"].str.lower().str.contains(search) |
metadata["title"].str.lower().str.contains(search),
"genre"] = "irish_folk"

In [105]:
sr["genre"] = "classic"

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [62]:
expanded_documents = [line.lower().replace("comp.", "composer").replace("compositeur", "composer") for line in documents]

tokenizer = RegexpTokenizer(r'\w+')

texts = [[unicodedata.normalize('NFKD', word).encode('ASCII', 'ignore') for word in tokenizer.tokenize(document.lower()) if (word not in stop_words and len(word) > 1)] for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

bigram = gensim.models.Phrases(texts)

texts_bigram = [bigram[text] for text in texts]
model_bigram = gensim.models.Word2Vec(texts_bigram, size=100, window=5, min_count=5, workers=4)

2016-10-18 16:10:29,263 : INFO : collecting all words and their counts
2016-10-18 16:10:29,263 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2016-10-18 16:10:30,394 : INFO : PROGRESS: at sentence #10000, processed 295344 words and 37796 word types
2016-10-18 16:10:31,713 : INFO : PROGRESS: at sentence #20000, processed 599965 words and 68838 word types
2016-10-18 16:10:34,723 : INFO : PROGRESS: at sentence #30000, processed 1281532 words and 410083 word types
2016-10-18 16:10:35,194 : INFO : collected 428645 word types from a corpus of 1378521 words (unigram + bigrams) and 32592 sentences
2016-10-18 16:10:35,194 : INFO : merging 428645 counts into Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2016-10-18 16:10:35,463 : INFO : merged Phrases<428645 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
2016-10-18 16:10:45,573 : INFO : collecting all words and their counts
2016-10-18 16:10:45,575 : INFO : PROGRESS: at sentence #0, pro